In [1]:
from PIL import Image
from torchvision import transforms
import os

In [3]:
import torch
# model = torch.hub.load('pytorch/vision:v0.9.0', 'googlenet', pretrained=True)
# model.eval()

Load data and data preprocess

In [7]:
#Load data: if Windows
path='Image/All_Cropped'

images={}
directory=['Mountain', 'Beach', 
           'Mug', 'Banana', 
           'Car', 'Plane', 
           'Lighthouse', 'Church']

for dir in directory:
    images[dir]=[file for file in os.listdir(path+'/'+dir) if file.endswith(('jpeg', 'jpg'))]

In [5]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
#type prob cate+filename
all_img={'type':[], 'typicality_score': [], 'dir':[]}

In [ ]:
for cate, filename in images.items():
  for f in filename:
    dir=os.path.join(path,cate,f)
    img=Image.open(dir)
    #print(dir)
    input_tensor=preprocess(img)
    input_batch = input_tensor.unsqueeze(0)

    if torch.cuda.is_available():
      input_batch= input_batch.to('cuda')
      model.to('cuda')

    with torch.no_grad():
      output = model(input_batch)

    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    typ_score=round(float(probabilities.max()*100),1)

    all_img['dir'].append(dir)
    all_img['typicality_score'].append(typ_score)
    if 'Mountain' in cate:
      all_img['type'].append('Mountain')
    if 'Beach' in cate:
      all_img['type'].append('Beach')
    if 'Mug' in cate:
      all_img['type'].append('Mug')
    if 'Banana' in cate:
      all_img['type'].append('Banana')
    if 'Car' in cate:
      all_img['type'].append('Car')
    if 'Plane' in cate:
      all_img['type'].append('Plane')
    if 'Lighthouse' in cate:
      all_img['type'].append('Lighthouse')
    if 'Church' in cate:
      all_img['type'].append('Church')

    #print('Typicality of ',f, ' = ',probabilities.max()*100)

In [ ]:
import pandas as pd
all_img_df=pd.DataFrame(all_img)
all_img_df.head()
#len(all_img_df)